# import

In [1]:
import numpy as np
import pandas as pd
import pandas.tseries.offsets as offsets
import datetime as dt
import time
import importlib
import logging
from retry import retry
#import traceback
#from retrying import retry
from sqlalchemy import create_engine
from sqlalchemy.types import Date, Integer, Float, Text
# from sqlalchemy.types import Integer
# from sqlalchemy.types import Text

import stock

In [ ]:
importlib.reload(stock)

# MySQLに接続

In [2]:
sql = stock.sql()

# 上場一覧から各種テーブルを作成 (メインはyahoo_stock_table)

In [3]:
# 東証のエクセルファイルを読み込む
all_stock_table = pd.read_excel('/Users/Really/Stockyard/_dl_data/data_j.xls') # http://www.jpx.co.jp/markets/statistics-equities/misc/01.html
all_stock_table.columns = ['date', 'code', 'name', 'market', 'code_33', 'category_33', 'code_17', 'category_17', 'code_scale', 'scale'] # 列名を変更

In [ ]:
all_stock_table

In [5]:
# marketの種別で集計
all_stock_table.groupby('market').count()

,date,code,name,code_33,category_33,code_17,category_17,code_scale,scale
market,,,,,,,,,
ETF・ETN,246,246,246,246,246,246,246,246,246
JASDAQ(グロース・内国株）,41,41,41,41,41,41,41,41,41
JASDAQ(スタンダード・内国株）,705,705,705,705,705,705,705,705,705
JASDAQ(スタンダード・外国株）,1,1,1,1,1,1,1,1,1
PRO Market,23,23,23,23,23,23,23,23,23
REIT・ベンチャーファンド・カントリーファンド・インフラファンド,64,64,64,64,64,64,64,64,64
マザーズ（内国株）,245,245,245,245,245,245,245,245,245
マザーズ（外国株）,1,1,1,1,1,1,1,1,1
出資証券,2,2,2,2,2,2,2,2,2


In [ ]:
# 上場一覧のテーブル保存
all_stock_table.to_csv('/Users/Really/Stockyard/_csv/all_stock_table.csv')
sql.write_table('all_stock_table', all_stock_table)
# all_stock_table.to_sql('all_stock_table', engine, if_exists='replace')

In [171]:
# PRO Marketを除いたテーブルの作成 (YahooにはPRO Marketのデータはない)
yahoo_stock_table = all_stock_table.ix[~all_stock_table['market'].str.contains('PRO Market')].reset_index(drop=True)

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
0,20171031,1301,極洋,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20171031,1305,ダイワ上場投信－トピックス,ETF・ETN,-,-,-,-,-,-
2,20171031,1306,ＴＯＰＩＸ連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
3,20171031,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
4,20171031,1309,上海株式指数・上証５０連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
5,20171031,1310,ダイワ上場投信－トピックス・コア３０,ETF・ETN,-,-,-,-,-,-
6,20171031,1311,ＴＯＰＩＸ Ｃｏｒｅ ３０ 連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
7,20171031,1312,ラッセル野村小型コア・インデックス連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
8,20171031,1313,サムスンＫＯＤＥＸ２００証券上場指数投資信託［株式］,ETF・ETN,-,-,-,-,-,-
9,20171031,1314,上場インデックスファンドＳ＆Ｐ日本新興株１００,ETF・ETN,-,-,-,-,-,-


In [ ]:
yahoo_stock_table

In [ ]:
# PRO Marketを除いたテーブルの保存
yahoo_stock_table.to_csv('/Users/Really/Stockyard/_csv/yahoo_stock_table.csv')
sql.write_table('yahoo_stock_table', yahoo_stock_table)
# yahoo_stock_table.to_sql('yahoo_stock_table', engine, if_exists='replace')

In [173]:
# 内国株のテーブル作成
domestic_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('内国株')].reset_index(drop=True)

In [ ]:
domestic_stock_table

In [ ]:
# 内国株のテーブル保存
domestic_stock_table.to_csv('/Users/Really/Stockyard/_csv/domestic_stock_table.csv')
sql.write_table('domestic_stock_table', domestic_stock_table)
# domestic_stock_table.to_sql('domestic_stock_table', engine, if_exists='replace')

In [202]:
# 内国株, PRO Market 以外のテーブル作成
ex_stock_table = yahoo_stock_table[~yahoo_stock_table['code'].isin(domestic_stock_table['code'])].reset_index(drop=True)

# 正規表現を使った書き方の例。
# ex_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('[^内国株）PRO Market]...$')].reset_index(drop=True)
# 文字列末尾の合致検索では、'$' の前の '.' の数で検索する文字数 ('.' * n + '$') が決定されているっぽい。
# つまりこの場合だと 'PRO Market' で実際に合致が確認されているのは末尾4文字の 'rket' 。
# '内国株）', 'PRO Market' をそれぞれグループ化するために' ()' で括る必要はないみたい。(ただし括っても同じ結果になる) 

# 下の書き方だと最後の1文字しか見ないことになるので、外国株も除外されてしまう。
# ex_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('[^内国株）PRO Market]$')].reset_index(drop=True)
# 上はつまり下の書き方と同じこと。
# ex_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('[^）t]$')].reset_index(drop=True)

In [203]:
ex_stock_table

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
0,20171031,1305,ダイワ上場投信－トピックス,ETF・ETN,-,-,-,-,-,-
1,20171031,1306,ＴＯＰＩＸ連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
2,20171031,1308,上場インデックスファンドＴＯＰＩＸ,ETF・ETN,-,-,-,-,-,-
3,20171031,1309,上海株式指数・上証５０連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
4,20171031,1310,ダイワ上場投信－トピックス・コア３０,ETF・ETN,-,-,-,-,-,-
5,20171031,1311,ＴＯＰＩＸ Ｃｏｒｅ ３０ 連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
6,20171031,1312,ラッセル野村小型コア・インデックス連動型上場投資信託,ETF・ETN,-,-,-,-,-,-
7,20171031,1313,サムスンＫＯＤＥＸ２００証券上場指数投資信託［株式］,ETF・ETN,-,-,-,-,-,-
8,20171031,1314,上場インデックスファンドＳ＆Ｐ日本新興株１００,ETF・ETN,-,-,-,-,-,-
9,20171031,1319,日経３００株価指数連動型上場投資信託,ETF・ETN,-,-,-,-,-,-


In [199]:
# marketの種別で集計
ex_stock_table.groupby('market').count()

,date,code,name,code_33,category_33,code_17,category_17,code_scale,scale
market,,,,,,,,,
ETF・ETN,246,246,246,246,246,246,246,246,246
JASDAQ(スタンダード・外国株）,1,1,1,1,1,1,1,1,1
REIT・ベンチャーファンド・カントリーファンド・インフラファンド,64,64,64,64,64,64,64,64,64
マザーズ（外国株）,1,1,1,1,1,1,1,1,1
出資証券,2,2,2,2,2,2,2,2,2
市場第一部（外国株）,3,3,3,3,3,3,3,3,3
市場第二部（外国株）,1,1,1,1,1,1,1,1,1


In [204]:
# 内国株, PRO Market 以外のテーブル保存
ex_stock_table.to_csv('/Users/Really/Stockyard/_csv/ex_stock_table.csv')
sql.write_table('ex_stock_table', ex_stock_table)
# ex_stock_table.to_sql('ex_stock_table', engine, if_exists='replace')

In [6]:
# 外国株のテーブル作成
foreign_stock_table = all_stock_table.ix[all_stock_table['market'].str.contains('外国株')].reset_index(drop=True)

In [7]:
foreign_stock_table

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
0,20171031,1773,ワイ・ティー・エル・コーポレーション・バーハッド,市場第一部（外国株）,2050,建設業,3,建設・資材,-,-
1,20171031,4875,メディシノバ・インク,JASDAQ(スタンダード・外国株）,3250,医薬品,5,医薬品,-,-
2,20171031,6697,テックポイント・インク,マザーズ（外国株）,3650,電気機器,9,電機・精密,-,-
3,20171031,8685,アメリカン・インターナショナル・グループ・インク,市場第一部（外国株）,7150,保険業,16,金融（除く銀行）,-,-
4,20171031,8686,アフラック・インコーポレーテッド,市場第一部（外国株）,7150,保険業,16,金融（除く銀行）,-,-
5,20171031,9399,新華ホールディングス・リミテッド（新華控股有限公司）,市場第二部（外国株）,5250,情報・通信業,10,情報通信・サービスその他,-,-


In [8]:
# 外国株のテーブル保存
foreign_stock_table.to_csv('/Users/Really/Stockyard/_csv/foreign_stock_table.csv')
sql.write_table('foreign_stock_table', foreign_stock_table)

In [207]:
# 型の確認
pd.DataFrame([
    sql.read_table('all_stock_table').dtypes,
    sql.read_table('yahoo_stock_table').dtypes,
    sql.read_table('domestic_stock_table').dtypes,
    sql.read_table('ex_stock_table').dtypes,
    sql.read_table('foreign_stock_table').dtypes],
    index=['all', 'yahoo', 'domestic', 'ex', 'foreign'])

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
all,int64,int64,object,object,object,object,object,object,object,object
yahoo,int64,int64,object,object,object,object,object,object,object,object
domestic,int64,int64,object,object,int64,object,int64,object,object,object
ex,int64,int64,object,object,object,object,object,object,object,object
foreign,int64,int64,object,object,int64,object,int64,object,object,object


# 上場一覧の更新

## TODO  
__次回12月分を処理する時にやり方を整理する__  
__new_stock_table を他のカーネルやスクリプトからも参照できるように保存する__

In [ ]:
# 旧テーブルの読み込み
all_stock_table_1710 = pd.read_excel('/Users/Really/Stockyard/_dl_data/data_j_1710.xls') # http://www.jpx.co.jp/markets/statistics-equities/misc/01.html
all_stock_table_1710.columns = ['date', 'code', 'name', 'market', 'code_33', 'category_33', 'code_17', 'category_17', 'code_scale', 'scale'] # 列名を変更
all_stock_table_1710

In [ ]:
# 旧テーブルの別名保存
all_stock_table_1710.to_csv('/Users/Really/Stockyard/_csv/all_stock_table_1710.csv')
sql.write_table('all_stock_table_1710', all_stock_table)
# all_stock_table_1710.to_sql('all_stock_table_1710', engine, if_exists='replace')

In [ ]:
# 新規銘柄のテーブル作成
new_stock_table = yahoo_stock_table[~yahoo_stock_table['code'].isin(all_stock_table_1710['code'])].reset_index(drop=True)
new_stock_table

# 上場廃止銘柄の更新

## TODO  
__次回12月分を処理する時に上場廃止銘柄のテーブル書き込みを追記仕様に書き換える__

In [ ]:
# 上場廃止銘柄のテーブル作成
discontinued_stock_table = all_stock_table_1710[~all_stock_table_1710['code'].isin(yahoo_stock_table['code'])].reset_index(drop=True)
# discontinued_stock_table = discontinued_stock_table.ix[~discontinued_stock_table['market'].str.contains('PRO Market')].reset_index(drop=True)
discontinued_stock_table

In [ ]:
# 上場廃止銘柄のテーブル保存
discontinued_stock_table.to_csv('/Users/Really/Stockyard/_csv/discontinued_stock_table.csv')
sql.write_table('discontinued_stock_table', all_stock_table)
# discontinued_stock_table.to_sql('discontinued_stock_table', engine, if_exists='replace')

# 連続読み込み用コードリストの作成例

## yahoo 連続読み込み用コードリスト作成

In [ ]:
start_index = 3100
increase_number = 100
end_index = start_index + increase_number

reading_code = sql.get_yahoo_stock_code(start_index, end_index)
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))

## 新規銘柄読み込み用コードリスト作成

In [ ]:
reading_code = list(new_stock_table['code'])
reading_code

# クラス不使用版コード

## MySQLに接続

In [ ]:
db_settings = {
    "host": 'localhost',
    "database": 'StockPrice_Yahoo_1',
    "user": 'user',
    "password": 'password',
    "port":'3306'
}
engine = create_engine('mysql://{user}:{password}@{host}:{port}/{database}'.format(**db_settings))

## 読み込む内国株のコードリスト作成 (クラス不使用版)

In [ ]:
# 内国株だけにする
# MySQLに保存済みの内国株テーブルから作成。今後はこちらを使用する
yahoo_stock_table = pd.read_sql_table('yahoo_stock_table', engine, index_col=None).drop('index', axis=1)

start_index = 2810
increase_number = 10
end_index = start_index + increase_number

reading_code = list(yahoo_stock_table['code'][start_index : end_index])
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))